In [ ]:
!pip install pydub

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import openai
import os
import csv
from pydub import AudioSegment
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

def transcribe_audio(api_key, audio_file):
    client = openai.OpenAI(api_key=api_key)

    with open(audio_file, "rb") as file:
        response = client.audio.transcriptions.create(
            model="whisper-1",
            file=file
        )

    return response.text  # Extract the transcribed text

def get_audio_duration(audio_file):
    """Returns the duration of the audio file in seconds."""
    audio = AudioSegment.from_file(audio_file)
    return len(audio) / 1000  # Convert milliseconds to seconds

def compute_speech_rate(audio_file, api_key):
    """Calculates the speech rate (WPS & WPM) of an audio file."""
    duration = get_audio_duration(audio_file)
    transcript = transcribe_audio(api_key, audio_file)

    word_count = len(transcript.split())
    wps = word_count / duration  # Words per second
    wpm = wps * 60  # Words per minute

    return round(wps, 2), round(wpm, 2)

def save_to_csv(results, output_file):
    """Save speech rate values to a CSV file in Google Drive."""
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["File Name", "Speech Rate (WPS)", "Speech Rate (WPM)"])
        writer.writerows(results)

def get_audio_files_from_drive(input_folder):
    """Get all audio files from a Google Drive folder."""
    files = []
    for file in os.listdir(input_folder):
        if file.endswith((".mp3", ".wav", ".flac")):  # Supports common audio formats
            files.append(os.path.join(input_folder, file))
    return files

def process_multiple_files(api_key, input_folder, output_file):
    results = []
    audio_files = get_audio_files_from_drive(input_folder)
    batch_size = 20  # Process 20 files at a time

    for i in range(0, len(audio_files), batch_size):
        batch = audio_files[i:i + batch_size]
        for audio_file in batch:
            print(f"Processing {audio_file}...")
            wps, wpm = compute_speech_rate(audio_file, api_key)
            results.append([os.path.basename(audio_file), wps, wpm])

        save_to_csv(results, output_file)
        print(f"Batch saved to {output_file}")

if __name__ == "__main__":
    API_KEY = ""  # Replace with your OpenAI API key
    INPUT_FOLDER = "/content/drive/MyDrive/audio_files"  # Folder in Google Drive containing audio files
    OUTPUT_FILE = "/content/drive/My Drive/Processed_Results/output.csv"  # Output CSV file path

    process_multiple_files(API_KEY, INPUT_FOLDER, OUTPUT_FILE)


MessageError: Error: credential propagation was unsuccessful